# Capstone Project-Suitable location for sushi restaurant in Toronto

### Introduction

##### Toronto is one of the most diverse city in the world. It is not only populated with diverse ethnicity group, but it is also an international financial center and a popuplar tourist attraction. Denamd for diverse range of food choice is hugh in Toronto and restaurant businness is an attractive investment. As a data scientist, my job is to analyze and provide solutions to potential investors.

#### Business Problem:

##### My client, Genki Sushi, which is a sushi chain originated in Japan (and has expanded to international cities like Hong Kong, Singapore, Australia, and USA etc) is interested in opening a branch in Toronto.  They would like to find out the demand for sushi restaurants in Toronto and the best location for opening their new sushi business.

### Data Section

#### Below 3 data source is to be used for the project:

##### 1) Demographic data in Toronto - to explore population of different ethnicity group in Toronto to analyze demand for sushi - we are making an assumption that areas that are densely populated with Japanese/Asian are in high sushi restaurant demand (https://en.wikipedia.org/wiki/Demographics_of_Toronto)
##### 2) List of postal codes of Toronto - for getting geo location data of different neighborhoods in Toronto for clustering different neighborhoods in Toronto (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)
##### 3) Get venue data from foursquare.com - using location data from 2) to find out distribution of sushi restaurants in Toronto, then we can cluster and analyze which location is the most popular for sushi restaurants  (https://api.foursquare.com)

#### Import neccessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

#### Import postal codes from 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' into pandas dataframe

In [2]:
pcode_list=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df_pcode=pcode_list[0]
print(df_pcode)

    Postcode           Borough  \
0        M1A      Not assigned   
1        M2A      Not assigned   
2        M3A        North York   
3        M4A        North York   
4        M5A  Downtown Toronto   
5        M5A  Downtown Toronto   
6        M6A        North York   
7        M6A        North York   
8        M7A      Queen's Park   
9        M8A      Not assigned   
10       M9A         Etobicoke   
11       M1B       Scarborough   
12       M1B       Scarborough   
13       M2B      Not assigned   
14       M3B        North York   
15       M4B         East York   
16       M4B         East York   
17       M5B  Downtown Toronto   
18       M5B  Downtown Toronto   
19       M6B        North York   
20       M7B      Not assigned   
21       M8B      Not assigned   
22       M9B         Etobicoke   
23       M9B         Etobicoke   
24       M9B         Etobicoke   
25       M9B         Etobicoke   
26       M9B         Etobicoke   
27       M1C       Scarborough   
28       M1C  

#### Import demographic data from https://en.wikipedia.org/wiki/Demographics_of_Toronto into pandas dataframe

In [3]:
demo_list=pd.read_html('https://en.wikipedia.org/wiki/Demographics_of_Toronto ')
df_demo=demo_list[5]
print(df_demo)

   Top 20 Ethnic origins in the City of Toronto  Population (2016)[33]  \
0                                       Chinese                 332830   
1                                       English                 331890   
2                                      Canadian                 323175   
3                                         Irish                 262965   
4                                      Scottish                 256250   
5                                   East Indian                 202675   
6                                       Italian                 182500   
7                                      Filipino                 162605   
8                                        German                 130900   
9                                        French                 122615   
10                                       Polish                 114530   
11                                   Portuguese                 100420   
12                                    

#### Get neccessary credentials for accessing foursquare.com

In [4]:
CLIENT_ID = '4JYYEXXQO4QR22TBLENKO2VI3QPUR5VWMAVXSXS21UAZHMAH' # your Foursquare ID
CLIENT_SECRET = 'ZSMHKPTVKK5OG2CTS4E1U5Y3GVJ5J3DGR0Y4FJ05GDUCD42A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4JYYEXXQO4QR22TBLENKO2VI3QPUR5VWMAVXSXS21UAZHMAH
CLIENT_SECRET:ZSMHKPTVKK5OG2CTS4E1U5Y3GVJ5J3DGR0Y4FJ05GDUCD42A
